In [2]:
# Load libraries

import xarray as xr
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import zarr
import gcsfs
import xesmf as xe
from scipy.interpolate import griddata

# Try accessing a dataset

#### Please read these first!:
More information on datasets here: https://computing-docs.readthedocs.io/en/latest/load_with_catalog.html 

This link contains more information on what CMIP6 data looks like and what all the labels mean: https://computing-docs.readthedocs.io/en/latest/download_cmip.html

Cristi has many model output files within his directory `/data/keeling/a/cristi/a/esm_data`. We're going to be using a lot of CMIP6 data, which is from the Climate Model Intercomparison Project phase 6. There's variable output from many different models under different scenarios within CMIP6. This data is kept in `/data/keeling/a/cristi/a/esm_data/cmip6`.

You can type `cd /data/keeling/a/cristi/a/esm_data/cmip6` into your terminal to look at the directory.

Once you're in the CMIP6 directory, you'll see all of the CMIP6 models. We'll be using data from any and all of the models, so feel free to pick whichever you'd like! I suggest using CESM2, CanESM5, BCC-CSM2-MR, or GFDL-ESM4 since they've given me the least trouble. 

Type `cd [modelname]` to go into that directory.

Once you're within a model directory, you'll see a bunch of experiments that the model has simulated.

The two experiments that we'll use the most are:
 - piControl - This is a control simulation, with everything kept to preindustrial conditions.
 - historical - This is a simulation with conditions following those in observations from 1850~2014. 

We'll be using piControl the most, so type `cd piControl`.

Typically you'll be taken to one or two subdirectories with a name like `r1i1p1f1`. Feel free to pick either one. This is the variant number, and we usually pick `r1i1p1f1`. 

Next, you'll be taken to the actual datasets! The first part of the file name is the variable. The variables we'll be using the most are:
 - tas - Surface air temperature. This is the air temperature just a couple meters off the ground.
 - ts - Surface skin temperature. This is the actual temperature of the surface.
 - rlds, rlut, rsds, hfls, hfss, rsdt, rsut - These are different radiative and heat fluxes. We'll go over this later.
 
Try following the load_with_catalog tutorial linked at the top to load a TAS dataset and then plot the time mean surface skin temperature over the whole globe.

In [4]:
# Following the tutorial up above, here's the catalog for the CMIP6 data

cat = pd.read_csv('/data/keeling/a/cristi/a/esm_data/cmip6_catalog.csv')
cat

,activity_id,branch_method,branch_time_in_child,branch_time_in_parent,experiment,experiment_id,frequency,grid,grid_label,institution_id,...,standard_name,long_name,units,vertical_levels,init_year,start_time,end_time,time_range,path,version
0,CMIP,standard,0.0,0.0,abrupt quadrupling of CO2,abrupt-4xCO2,mon,native atmosphere N96 grid (144x192 latxlon),gn,CSIRO-ARCCSS,...,surface_upward_latent_heat_flux,Surface Upward Latent Heat Flux,W m-2,1.0,NaN,0950-01-16 12:00:00,1099-12-16 12:00:00,0950-01-16 12:00:00-1099-12-16 12:00:00,/data/cristi/a/cristi/esm_data/cmip6/ACCESS-CM...,v0
1,CMIP,standard,0.0,0.0,abrupt quadrupling of CO2,abrupt-4xCO2,mon,native atmosphere N96 grid (144x192 latxlon),gn,CSIRO-ARCCSS,...,surface_upward_sensible_heat_flux,Surface Upward Sensible Heat Flux,W m-2,1.0,NaN,0950-01-16 12:00:00,1099-12-16 12:00:00,0950-01-16 12:00:00-1099-12-16 12:00:00,/data/cristi/a/cristi/esm_data/cmip6/ACCESS-CM...,v0
2,CMIP,standard,0.0,0.0,abrupt quadrupling of CO2,abrupt-4xCO2,mon,native atmosphere N96 grid (144x192 latxlon),gn,CSIRO-ARCCSS,...,surface_downwelling_longwave_flux_in_air,Surface Downwelling Longwave Radiation,W m-2,1.0,NaN,0950-01-16 12:00:00,1099-12-16 12:00:00,0950-01-16 12:00:00-1099-12-16 12:00:00,/data/cristi/a/cristi/esm_data/cmip6/ACCESS-CM...,v0
3,CMIP,standard,0.0,0.0,abrupt quadrupling of CO2,abrupt-4xCO2,mon,native atmosphere N96 grid (144x192 latxlon),gn,CSIRO-ARCCSS,...,surface_upwelling_longwave_flux_in_air,Surface Upwelling Longwave Radiation,W m-2,1.0,NaN,0950-01-16 12:00:00,1099-12-16 12:00:00,0950-01-16 12:00:00-1099-12-16 12:00:00,/data/cristi/a/cristi/esm_data/cmip6/ACCESS-CM...,v0
4,CMIP,standard,0.0,0.0,abrupt quadrupling of CO2,abrupt-4xCO2,mon,native atmosphere N96 grid (144x192 latxlon),gn,CSIRO-ARCCSS,...,toa_outgoing_longwave_flux,TOA Outgoing Longwave Radiation,W m-2,1.0,NaN,0950-01-16 12:00:00,1099-12-16 12:00:00,0950-01-16 12:00:00-1099-12-16 12:00:00,/data/cristi/a/cristi/esm_data/cmip6/ACCESS-CM...,v0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,ScenarioMIP,standard,59400.0,59400.0,update of RCP8.5 based on SSP5,ssp585,mon,Native N96 grid; 192 x 144 longitude/latitude,gn,MOHC,...,surface_downwelling_shortwave_flux_in_air,Surface Downwelling Shortwave Radiation,W m-2,1.0,NaN,2015-01-16 00:00:00,2100-12-16 00:00:00,2015-01-16 00:00:00-2100-12-16 00:00:00,/data/cristi/a/cristi/esm_data/cmip6/UKESM1-0-...,v0
11610,ScenarioMIP,standard,59400.0,59400.0,update of RCP8.5 based on SSP5,ssp585,mon,Native N96 grid; 192 x 144 longitude/latitude,gn,MOHC,...,toa_incoming_shortwave_flux,TOA Incident Shortwave Radiation,W m-2,1.0,NaN,2015-01-16 00:00:00,2100-12-16 00:00:00,2015-01-16 00:00:00-2100-12-16 00:00:00,/data/cristi/a/cristi/esm_data/cmip6/UKESM1-0-...,v0
11611,ScenarioMIP,standard,59400.0,59400.0,update of RCP8.5 based on SSP5,ssp585,mon,Native N96 grid; 192 x 144 longitude/latitude,gn,MOHC,...,surface_upwelling_shortwave_flux_in_air,Surface Upwelling Shortwave Radiation,W m-2,1.0,NaN,2015-01-16 00:00:00,2100-12-16 00:00:00,2015-01-16 00:00:00-2100-12-16 00:00:00,/data/cristi/a/cristi/esm_data/cmip6/UKESM1-0-...,v0
11612,ScenarioMIP,standard,59400.0,59400.0,update of RCP8.5 based on SSP5,ssp585,mon,Native N96 grid; 192 x 144 longitude/latitude,gn,MOHC,...,toa_outgoing_shortwave_flux,TOA Outgoing Shortwave Radiation,W m-2,1.0,NaN,2015-01-16 00:00:00,2100-12-16 00:00:00,2015-01-16 00:00:00-2100-12-16 00:00:00,/data/cristi/a/cristi/esm_data/cmip6/UKESM1-0-...,v0


As an example to load data, try looking at the example below. 

This example takes rsdt data from CESM2 run at monthly intervals for CMIP6. It also only returns the first dataset that corresponds to what we asked for (head(1)).

In [5]:
path_rsdt_control = cat.loc[(cat['variable_id']=='rsdt') &
               (cat['activity_id']=='CMIP') &
               (cat['frequency']=='mon') & 
               (cat['source_id']=='CESM2') &
               (cat['experiment_id'].str.contains('Control'))].head(1)['path'].to_list()
path_rsdt_control

['/data/cristi/a/cristi/esm_data/cmip6/CESM2/piControl/r1i1p1f1/rsdt_Amon_CESM2_piControl_r1i1p1f1_gn_000101-009912.nc']

In [11]:
ds = xr.open_dataset(path_rsdt_control[0])
ds

<xarray.Dataset>
Dimensions:    (time: 1188, lat: 192, lon: 288, nbnd: 2)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time       (time) object 0001-01-15 12:00:00 ... 0099-12-15 12:00:00
Dimensions without coordinates: nbnd
Data variables:
    rsdt       (time, lat, lon) float32 ...
    time_bnds  (time, nbnd) object ...
    lat_bnds   (lat, nbnd) float32 ...
    lon_bnds   (lon, nbnd) float32 ...
Attributes: (12/45)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    case_id:                3
    cesm_casename:          b.e21.B1850.f09_g17.CMIP6-piControl.001
    contact:                cesm_cmip6@ucar.edu
    creation_date:          2019-01-20T21:36:37Z
    ...                     ...
    variable_id:            rsdt
    variant_info:           CMIP6 CESM2 piControl experiment with CAM6, inter...
    variant_label:          r1i1p1f1
    branch_time_in_parent:  48545.0
    branch_time_in_child:   0.0
    branch_method:          standard

# Skin Temperature

Try doing something similar for skin temperature! Then, take a mean over time in order to get mean surface temperature over the whole globe, and then plot the corresponding map.

# TOA Radiation

After reading through the notes of atmospheric radiation, try doing the same for TOA radiation! 

You will need to:
 - Pick out a model
 - Select the appropriate variables for that model
 - Add or subtract these variables from each other accordingly
 
And then you can continue the same process as you did for surface temperature.

# Surface Fluxes

Try doing the same for the surface fluxes! Use the "Radiation introduction" slideshow as a reference and read the pdf on latent heat flux and sensible heat flux to get an understanding of what's happening.

# Regression

You'll be doing a regression over time of two different variables at the same location! The following steps should help guide you through this.

### Step 1: Load your skin temperature and TOA Radiation datasets

You've already done this, so this should be pretty simple. I'd suggest assigning TOA to a variable (e.g. TOA = ds_rsdt.rsdt - ds_rsut.rsut - ds_rlut.rlut)

### Step 2: Pick a location on Earth

You'll want to pick a latitude and longitude to do this regression at. In the following example, I'll use a location at the equator (lat=0) with longitude=180.

TOA_equator = TOA.sel(lat=0, method='nearest').isel(lon=180, method='nearest')

sel picks a value from the data array based on the label. So, it picks a location at latitude 0 (On the equator) and longitude 180 (Somewhere in the Pacific Ocean).

method='nearest' picks the closest location we have to that longitude or latitude. So, if we don't have latitude 0 but have latitude 0.5 instead, it'll pick lat=0.5.

### Step 3: Get the anomalies from both datasets.

In order to get an understanding of what anomalies and climatologies are, I'd suggest skimming over this webpage: https://cds.climate.copernicus.eu/toolbox/doc/how-to/13_how_to_calculate_climatologies_and_anomalies/13_how_to_calculate_climatologies_and_anomalies.html

In short, climatologies are the averages of variables over different time periods, while anomalies are the differences between the actual values and these averages. We're calculating monthly anomalies, so we want values that are the difference between our monthly values and what the normal average value for that month is.

For example, if February this year was around 50 degrees on average, but Februaries are typically 40 degrees, that means the February anomaly this year was 10 degrees.

Calculate the monthly climatologies using groupby means on months.

climatology_sst = sst.groupby('time.month').mean('time')

Then get anomalies by subtracting our actual values by the climatologies:

sst_anomalies = sst.groupby('time.month') - climatology_sst


### Step 4: Do the regression!

Now we can do the regression! I'm not sure how to use LinearRegression exactly, but I suggest using fit(sst, TOA). You should get the regression coefficient. Let me know if this works, and we can talk out a solution if it doesn't (or use np.polyfit).

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

### Step 5: Do the same with SST and Surface Radiation (If everything works out)